# PointNet


In [1]:
import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
import tensorflow as tf
import open3d.ml as _ml3d
import open3d.ml.tf as ml3d
import open3d as o3d
import numpy as np
import json
tf.random.set_seed(1234)

In [19]:
NUM_POINTS = 4096
NUM_CLASSES = 4
BATCH_SIZE = 16

In [3]:
# points = mesh.sample(2048)

# fig = plt.figure(figsize=(5, 5))
# ax = fig.add_subplot(111, projection="3d")
# ax.scatter(points[:, 0], points[:, 1], points[:, 2])
# ax.set_axis_off()
# plt.show()

In [4]:
gt_pcl_path = ''
gt_lbl_path = ''
filenames_gt_pcl = sorted(os.listdir(gt_pcl_path))
filenames_lbl_pcl = sorted(os.listdir(gt_lbl_path))

pcl_bbox_list = []
for gt_pcl, gt_lbl in zip(filenames_gt_pcl, filenames_lbl_pcl):
    pcl = o3d.io.read_point_cloud(os.path.join(gt_pcl_path, gt_pcl))
    pcl_lbl = json.load(open(os.path.join(gt_lbl_path, gt_lbl), 'rb'))
    bboxes_descr = pcl_lbl['figures']
    objects_descr = pcl_lbl['objects']
    pcl_bbox_list_inl = []
    for bboxd in bboxes_descr:
        obj_key = bboxd['objectKey']
        bbox3d_pos_x = bboxd['geometry']['position']['x']
        bbox3d_pos_y = bboxd['geometry']['position']['y']
        bbox3d_pos_z = bboxd['geometry']['position']['z']
        bbox3d_dim_x = bboxd['geometry']['dimensions']['x']
        bbox3d_dim_y = bboxd['geometry']['dimensions']['y']
        bbox3d_dim_z = bboxd['geometry']['dimensions']['z']
        x1 = bbox3d_pos_x - bbox3d_dim_x / 2
        x2 = bbox3d_pos_x + bbox3d_dim_x / 2
        y1 = bbox3d_pos_y - bbox3d_dim_y / 2
        y2 = bbox3d_pos_y + bbox3d_dim_y / 2
        z1 = bbox3d_pos_z - bbox3d_dim_z / 2
        z2 = bbox3d_pos_z + bbox3d_dim_z / 2
        min_bound = (min(x1, x2), min(y1, y2), min(z1, z2))
        max_bound = (max(x1, x2), max(y1, y2), max(z1, z2))
        # print(min_bound, max_bound)
        bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=min_bound,max_bound=max_bound)
        bbox_pcl = pcl.crop(bbox)
        # print(np.asarray(bbox_pcl.points))
        # o3d.visualization.draw_geometries([pcl])
        lbl_name = [obj['classTitle'] for obj in objects_descr if obj['key'] == obj_key][0]
        pcl_bbox_list_inl.append((bbox, bbox_pcl, lbl_name))
    pcl_bbox_list.append((pcl, pcl_bbox_list_inl))

In [5]:
o3d.visualization.draw_geometries([pcl_bbox_list[1][0], pcl_bbox_list[1][1][0][0], pcl_bbox_list[1][1][0][1],
pcl_bbox_list[1][1][1][0], pcl_bbox_list[1][1][1][1], pcl_bbox_list[1][1][2][0], pcl_bbox_list[1][1][2][1]])

IndexError: list index out of range

In [6]:
pcl_bbox_list

, 0.668912, 2.23856), max: (-1.23604, 1.12384, 3.1512),
    PointCloud with 12703 points.,
    'human')]),
 (PointCloud with 752375 points.,
  [(AxisAlignedBoundingBox: min: (-0.569375, -0.164108, 1.29324), max: (0.0388805, 0.841746, 3.13712),
    PointCloud with 25301 points.,
    'human'),
   (AxisAlignedBoundingBox: min: (-0.680231, -0.625057, 1.53091), max: (-0.415826, -0.100473, 2.45828),
    PointCloud with 8529 points.,
    'wear')]),
 (PointCloud with 749647 points.,
  [(AxisAlignedBoundingBox: min: (-0.881959, -0.340444, 2.27918), max: (-0.119362, 0.393125, 3.12792),
    PointCloud with 23559 points.,
    'human')]),
 (PointCloud with 761381 points.,
  [(AxisAlignedBoundingBox: min: (-0.987228, -0.255525, 1.36277), max: (-0.149402, 0.321572, 3.1128),
    PointCloud with 25320 points.,
    'human'),
   (AxisAlignedBoundingBox: min: (-0.662418, -0.467941, 1.77886), max: (-0.369402, -0.19359, 2.76957),
    PointCloud with 6276 points.,
    'wear')]),
 (PointCloud with 753010 poin

In [478]:
pcl_lbl

{'description': '',
 'key': 'e4323724dacf439e84b910601949e934',
 'tags': [],
 'objects': [{'key': '4757bbc2dc6940a8b4315a1ed9cace1b',
   'classTitle': 'human',
   'tags': [],
   'labelerLogin': 'suhanovroman',
   'updatedAt': '2021-05-22T10:14:46.039Z',
   'createdAt': '2021-05-22T10:14:46.039Z'},
  {'key': '7605150907f44534947c87ab0f0cc30d',
   'classTitle': 'other',
   'tags': [],
   'labelerLogin': 'suhanovroman',
   'updatedAt': '2021-05-22T10:23:16.316Z',
   'createdAt': '2021-05-22T10:23:16.316Z'}],
 'figures': [{'key': '78e11b21f93a4c64853c21b946f0d70c',
   'objectKey': '4757bbc2dc6940a8b4315a1ed9cace1b',
   'geometryType': 'cuboid_3d',
   'geometry': {'position': {'x': 0.36954893856172943,
     'y': 0.07402263375818492,
     'z': 2.33711201307192},
    'rotation': {'x': 0, 'y': 0, 'z': 0},
    'dimensions': {'x': 0.6239018898971955,
     'y': 0.5818194646144317,
     'z': 1.6415971600966917}},
   'labelerLogin': 'suhanovroman',
   'updatedAt': '2021-05-22T14:24:40.501Z',
   'cr

In [8]:
bboxpcl_class = []
for i, obj in enumerate(pcl_bbox_list):
    for j, obj2 in enumerate(obj[1]):
        # print(obj2[0], obj2[1], obj2[2])
        bboxpcl_class.append((obj2[1], obj2[2]))

In [9]:
classes = ['human', 'wear', 'limb', 'other']
train_points = [np.asarray(obj[0].points) for obj in bboxpcl_class]
train_points2 = []
for tp in train_points:
    p_indexes = np.random.choice(np.arange(len(tp)), size=NUM_POINTS)
    train_points2.append(tp[p_indexes])

train_labels = np.array([classes.index(obj[1]) for obj in bboxpcl_class])
train_points = np.array(train_points2)

In [11]:
len(train_labels)

196

In [74]:
def prepare_dataset(data, num_points=2048):
    pass

In [20]:

# def augment(points, label):
#     # jitter points
#     points += tf.random.uniform(points.shape, -0.005, 0.005, dtype=tf.float64)
#     # shuffle points
#     points = tf.random.shuffle(points)
#     return points.numpy(), label


# # train_dataset = tf.data.Dataset.from_tensor_slices((train_points, train_labels))
# # test_dataset = tf.data.Dataset.from_tensor_slices((test_points, test_labels))

# # train_dataset = train_dataset.shuffle(len(train_points)).map(augment).batch(BATCH_SIZE)
# # test_dataset = test_dataset.shuffle(len(test_points)).batch(BATCH_SIZE)

# def gen_fn(train_points, train_labels):
#     def gen():
#         for i in range(len(train_points)):
#             points, labels = augment(train_points[i], train_labels[i])
#             yield np.array([points]), np.array([labels])
#     return gen()

# def inf_gen_fn(train_points, train_labels):
#     def inf_gen():
#         while True:
#             for i in range(len(train_points)):
#                 points, labels = augment(train_points[i], train_labels[i])
#                 p_indexes = np.random.choice(np.arange(len(points)), size=NUM_POINTS)
#                 points = points[p_indexes]
#                 yield np.array([points]), np.array([labels])
#     return inf_gen()

def augment(points, label):
    # jitter points
    points += tf.random.uniform(points.shape, -0.01, 0.01, dtype=tf.float64)
    # shuffle points
    points = tf.random.shuffle(points)
    return points, label


train_dataset = tf.data.Dataset.from_tensor_slices((train_points, train_labels))
# test_dataset = tf.data.Dataset.from_tensor_slices((test_points, test_labels))

train_dataset = train_dataset.shuffle(len(train_points)).map(augment).batch(BATCH_SIZE)
# test_dataset = test_dataset.shuffle(len(test_points)).batch(BATCH_SIZE)

In [21]:
train_dataset

<BatchDataset shapes: ((None, 4096, 3), (None,)), types: (tf.float64, tf.int64)>

In [22]:
# train_gen = inf_gen_fn(train_points, train_labels)

In [23]:
def conv_bn(x, filters):
    x = layers.Conv1D(filters, kernel_size=1, padding="valid")(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)


def dense_bn(x, filters):
    x = layers.Dense(filters)(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)


In [24]:

class OrthogonalRegularizer(keras.regularizers.Regularizer):
    def __init__(self, num_features, l2reg=0.001):
        self.num_features = num_features
        self.l2reg = l2reg
        self.eye = tf.eye(num_features)

    def __call__(self, x):
        x = tf.reshape(x, (-1, self.num_features, self.num_features))
        xxt = tf.tensordot(x, x, axes=(2, 2))
        xxt = tf.reshape(xxt, (-1, self.num_features, self.num_features))
        return tf.reduce_sum(self.l2reg * tf.square(xxt - self.eye))


 We can then define a general function to build T-net layers.


In [25]:
def tnet(inputs, num_features):

    # Initalise bias as the indentity matrix
    bias = keras.initializers.Constant(np.eye(num_features).flatten())
    reg = OrthogonalRegularizer(num_features)

    x = conv_bn(inputs, 32)
    x = conv_bn(x, 64)
    x = conv_bn(x, 512)
    x = layers.GlobalMaxPooling1D()(x)
    x = dense_bn(x, 256)
    x = dense_bn(x, 128)
    x = layers.Dense(
        num_features * num_features,
        kernel_initializer="zeros",
        bias_initializer=bias,
        activity_regularizer=reg,
    )(x)
    feat_T = layers.Reshape((num_features, num_features))(x)
    # Apply affine transformation to input features
    return layers.Dot(axes=(2, 1))([inputs, feat_T])


In [26]:
inputs = keras.Input(shape=(NUM_POINTS, 3))

x = tnet(inputs, 3)
x = conv_bn(x, 32)
x = conv_bn(x, 32)
x = tnet(x, 32)
x = conv_bn(x, 64)
x = conv_bn(x, 64)
x = conv_bn(x, 512)
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, 256)
x = layers.Dropout(0.6)(x)
x = dense_bn(x, 128)
x = layers.Dropout(0.6)(x)

outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet")
model.summary()

Model: "pointnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 4096, 3)]    0                                            
__________________________________________________________________________________________________
conv1d_11 (Conv1D)              (None, 4096, 32)     128         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_17 (BatchNo (None, 4096, 32)     128         conv1d_11[0][0]                  
__________________________________________________________________________________________________
activation_17 (Activation)      (None, 4096, 32)     0           batch_normalization_17[0][0]     
___________________________________________________________________________________________

In [27]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(),
    metrics=["sparse_categorical_accuracy"],
)

model.fit(train_dataset, epochs=5)

Epoch 1/5
13/13 [==============================] - 17s 1s/step - loss: 3.0841 - sparse_categorical_accuracy: 0.3520
Epoch 2/5
13/13 [==============================] - 17s 1s/step - loss: 2.1938 - sparse_categorical_accuracy: 0.5510
Epoch 3/5
13/13 [==============================] - 17s 1s/step - loss: 1.6606 - sparse_categorical_accuracy: 0.6684
Epoch 4/5
13/13 [==============================] - 17s 1s/step - loss: 1.5499 - sparse_categorical_accuracy: 0.7194
Epoch 5/5
13/13 [==============================] - 17s 1s/step - loss: 1.4509 - sparse_categorical_accuracy: 0.7296


In [29]:
model.fit(train_dataset, epochs=10)

Epoch 1/10
13/13 [==============================] - 17s 1s/step - loss: 1.0708 - sparse_categorical_accuracy: 0.8010
Epoch 2/10
13/13 [==============================] - 17s 1s/step - loss: 1.3591 - sparse_categorical_accuracy: 0.7602
Epoch 3/10
13/13 [==============================] - 17s 1s/step - loss: 1.3639 - sparse_categorical_accuracy: 0.7908
Epoch 4/10
13/13 [==============================] - 17s 1s/step - loss: 1.2977 - sparse_categorical_accuracy: 0.7653
Epoch 5/10
13/13 [==============================] - 17s 1s/step - loss: 1.2162 - sparse_categorical_accuracy: 0.7653
Epoch 6/10
13/13 [==============================] - 17s 1s/step - loss: 1.1998 - sparse_categorical_accuracy: 0.8061
Epoch 7/10
13/13 [==============================] - 17s 1s/step - loss: 1.2507 - sparse_categorical_accuracy: 0.7602
Epoch 8/10
13/13 [==============================] - 17s 1s/step - loss: 1.2274 - sparse_categorical_accuracy: 0.7959
Epoch 9/10
13/13 [==============================] - 17s 1s/step 

In [429]:
bboxpcl_class

[(PointCloud with 28816 points., 'human'),
 (PointCloud with 20184 points., 'human'),
 (PointCloud with 29876 points., 'human'),
 (PointCloud with 4626 points., 'wear'),
 (PointCloud with 40868 points., 'human'),
 (PointCloud with 22442 points., 'human'),
 (PointCloud with 5554 points., 'wear'),
 (PointCloud with 3514 points., 'other'),
 (PointCloud with 49202 points., 'human'),
 (PointCloud with 6957 points., 'other'),
 (PointCloud with 31477 points., 'human'),
 (PointCloud with 25320 points., 'human'),
 (PointCloud with 6276 points., 'wear'),
 (PointCloud with 42044 points., 'human'),
 (PointCloud with 3149 points., 'limb'),
 (PointCloud with 35464 points., 'human'),
 (PointCloud with 917 points., 'other'),
 (PointCloud with 2931 points., 'wear'),
 (PointCloud with 25346 points., 'human'),
 (PointCloud with 5526 points., 'other'),
 (PointCloud with 18855 points., 'human'),
 (PointCloud with 14672 points., 'human'),
 (PointCloud with 4427 points., 'human'),
 (PointCloud with 43086 poi

In [464]:
pred = model.predict(np.asarray([train_points[0]], np.float32))
pred.argmax()

0

In [30]:
model.save_weights('./pointnet_weights2.h5')

In [472]:
model2.load_weights('./pointnet_weights.h5')

In [474]:
pred2 = model2.predict(np.asarray([train_points[12]], np.float32))
pred2.argmax()

3